In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
X_train = pd.read_csv('datasets/titanic/train.csv')
X_test = pd.read_csv('datasets/titanic/test.csv')
y_train = X_train['Survived']

In [4]:
X_train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [5]:
# Definition of the CategoricalEncoder class, copied from PR #9151.
# Just run this cell, or copy it to your code, do not try to understand it (yet).

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils import check_array
from sklearn.preprocessing import LabelEncoder
from scipy import sparse

class CategoricalEncoder(BaseEstimator, TransformerMixin):
    """Encode categorical features as a numeric array.
    The input to this transformer should be a matrix of integers or strings,
    denoting the values taken on by categorical (discrete) features.
    The features can be encoded using a one-hot aka one-of-K scheme
    (``encoding='onehot'``, the default) or converted to ordinal integers
    (``encoding='ordinal'``).
    This encoding is needed for feeding categorical data to many scikit-learn
    estimators, notably linear models and SVMs with the standard kernels.
    Read more in the :ref:`User Guide <preprocessing_categorical_features>`.
    Parameters
    ----------
    encoding : str, 'onehot', 'onehot-dense' or 'ordinal'
        The type of encoding to use (default is 'onehot'):
        - 'onehot': encode the features using a one-hot aka one-of-K scheme
          (or also called 'dummy' encoding). This creates a binary column for
          each category and returns a sparse matrix.
        - 'onehot-dense': the same as 'onehot' but returns a dense array
          instead of a sparse matrix.
        - 'ordinal': encode the features as ordinal integers. This results in
          a single column of integers (0 to n_categories - 1) per feature.
    categories : 'auto' or a list of lists/arrays of values.
        Categories (unique values) per feature:
        - 'auto' : Determine categories automatically from the training data.
        - list : ``categories[i]`` holds the categories expected in the ith
          column. The passed categories are sorted before encoding the data
          (used categories can be found in the ``categories_`` attribute).
    dtype : number type, default np.float64
        Desired dtype of output.
    handle_unknown : 'error' (default) or 'ignore'
        Whether to raise an error or ignore if a unknown categorical feature is
        present during transform (default is to raise). When this is parameter
        is set to 'ignore' and an unknown category is encountered during
        transform, the resulting one-hot encoded columns for this feature
        will be all zeros.
        Ignoring unknown categories is not supported for
        ``encoding='ordinal'``.
    Attributes
    ----------
    categories_ : list of arrays
        The categories of each feature determined during fitting. When
        categories were specified manually, this holds the sorted categories
        (in order corresponding with output of `transform`).
    Examples
    --------
    Given a dataset with three features and two samples, we let the encoder
    find the maximum value per feature and transform the data to a binary
    one-hot encoding.
    >>> from sklearn.preprocessing import CategoricalEncoder
    >>> enc = CategoricalEncoder(handle_unknown='ignore')
    >>> enc.fit([[0, 0, 3], [1, 1, 0], [0, 2, 1], [1, 0, 2]])
    ... # doctest: +ELLIPSIS
    CategoricalEncoder(categories='auto', dtype=<... 'numpy.float64'>,
              encoding='onehot', handle_unknown='ignore')
    >>> enc.transform([[0, 1, 1], [1, 0, 4]]).toarray()
    array([[ 1.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.],
           [ 0.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.]])
    See also
    --------
    sklearn.preprocessing.OneHotEncoder : performs a one-hot encoding of
      integer ordinal features. The ``OneHotEncoder assumes`` that input
      features take on values in the range ``[0, max(feature)]`` instead of
      using the unique values.
    sklearn.feature_extraction.DictVectorizer : performs a one-hot encoding of
      dictionary items (also handles string-valued features).
    sklearn.feature_extraction.FeatureHasher : performs an approximate one-hot
      encoding of dictionary items or strings.
    """

    def __init__(self, encoding='onehot', categories='auto', dtype=np.float64,
                 handle_unknown='error'):
        self.encoding = encoding
        self.categories = categories
        self.dtype = dtype
        self.handle_unknown = handle_unknown

    def fit(self, X, y=None):
        """Fit the CategoricalEncoder to X.
        Parameters
        ----------
        X : array-like, shape [n_samples, n_feature]
            The data to determine the categories of each feature.
        Returns
        -------
        self
        """

        if self.encoding not in ['onehot', 'onehot-dense', 'ordinal']:
            template = ("encoding should be either 'onehot', 'onehot-dense' "
                        "or 'ordinal', got %s")
            raise ValueError(template % self.handle_unknown)

        if self.handle_unknown not in ['error', 'ignore']:
            template = ("handle_unknown should be either 'error' or "
                        "'ignore', got %s")
            raise ValueError(template % self.handle_unknown)

        if self.encoding == 'ordinal' and self.handle_unknown == 'ignore':
            raise ValueError("handle_unknown='ignore' is not supported for"
                             " encoding='ordinal'")

        X = check_array(X, dtype=np.object, accept_sparse='csc', copy=True)
        n_samples, n_features = X.shape

        self._label_encoders_ = [LabelEncoder() for _ in range(n_features)]

        for i in range(n_features):
            le = self._label_encoders_[i]
            Xi = X[:, i]
            if self.categories == 'auto':
                le.fit(Xi)
            else:
                valid_mask = np.in1d(Xi, self.categories[i])
                if not np.all(valid_mask):
                    if self.handle_unknown == 'error':
                        diff = np.unique(Xi[~valid_mask])
                        msg = ("Found unknown categories {0} in column {1}"
                               " during fit".format(diff, i))
                        raise ValueError(msg)
                le.classes_ = np.array(np.sort(self.categories[i]))

        self.categories_ = [le.classes_ for le in self._label_encoders_]

        return self

    def transform(self, X):
        """Transform X using one-hot encoding.
        Parameters
        ----------
        X : array-like, shape [n_samples, n_features]
            The data to encode.
        Returns
        -------
        X_out : sparse matrix or a 2-d array
            Transformed input.
        """
        X = check_array(X, accept_sparse='csc', dtype=np.object, copy=True)
        n_samples, n_features = X.shape
        X_int = np.zeros_like(X, dtype=np.int)
        X_mask = np.ones_like(X, dtype=np.bool)

        for i in range(n_features):
            valid_mask = np.in1d(X[:, i], self.categories_[i])

            if not np.all(valid_mask):
                if self.handle_unknown == 'error':
                    diff = np.unique(X[~valid_mask, i])
                    msg = ("Found unknown categories {0} in column {1}"
                           " during transform".format(diff, i))
                    raise ValueError(msg)
                else:
                    # Set the problematic rows to an acceptable value and
                    # continue `The rows are marked `X_mask` and will be
                    # removed later.
                    X_mask[:, i] = valid_mask
                    X[:, i][~valid_mask] = self.categories_[i][0]
            X_int[:, i] = self._label_encoders_[i].transform(X[:, i])

        if self.encoding == 'ordinal':
            return X_int.astype(self.dtype, copy=False)

        mask = X_mask.ravel()
        n_values = [cats.shape[0] for cats in self.categories_]
        n_values = np.array([0] + n_values)
        indices = np.cumsum(n_values)

        column_indices = (X_int + indices[:-1]).ravel()[mask]
        row_indices = np.repeat(np.arange(n_samples, dtype=np.int32),
                                n_features)[mask]
        data = np.ones(n_samples * n_features)[mask]

        out = sparse.csc_matrix((data, (row_indices, column_indices)),
                                shape=(n_samples, indices[-1]),
                                dtype=self.dtype).tocsr()
        if self.encoding == 'onehot-dense':
            return out.toarray()
        else:
            return out

Step 1: Remove useless columns

In [6]:
from sklearn.base import BaseEstimator, TransformerMixin

class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        return X[self.attribute_names].values

In [7]:
from sklearn.preprocessing import StandardScaler, Imputer, LabelEncoder
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn_pandas import DataFrameMapper, CategoricalImputer

num_pipeline = Pipeline([
    ('selector', DataFrameSelector(['SibSp', 'Parch', 'Fare', 'Age'])),
    ('median_imputer', Imputer(strategy='median')),
    ('standard_scaler', StandardScaler())
])
cat_pipeline = Pipeline([
    ('selector', DataFrameSelector(['Sex', 'Pclass', 'Embarked'])),
    ('categorical_imputer', CategoricalImputer()),
    ('binary_encoder', CategoricalEncoder(encoding='onehot-dense')),
])
full_pipeline = FeatureUnion(transformer_list=[
    ('num_pipeline', num_pipeline),
    ('cat_pipeline', cat_pipeline),
])

In [8]:
X_train = full_pipeline.fit_transform(X_train)
X_train

array([[ 0.43279337, -0.47367361, -0.50244517, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.43279337, -0.47367361,  0.78684529, ...,  1.        ,
         0.        ,  0.        ],
       [-0.4745452 , -0.47367361, -0.48885426, ...,  0.        ,
         0.        ,  1.        ],
       ..., 
       [ 0.43279337,  2.00893337, -0.17626324, ...,  0.        ,
         0.        ,  1.        ],
       [-0.4745452 , -0.47367361, -0.04438104, ...,  1.        ,
         0.        ,  0.        ],
       [-0.4745452 , -0.47367361, -0.49237783, ...,  0.        ,
         1.        ,  0.        ]])

Step 3: Search for fitting model

In [9]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
param_grid = [
    {'min_samples_split': [2, 4, 6, 8, 10, 20, 40, 60, 80], 'max_features': [2, 3, 6, 8, 10]},
]
tree_classifier = DecisionTreeClassifier()
grid_search = GridSearchCV(tree_classifier, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)
print('Best params:', grid_search.best_params_)

cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print("Score:", mean_score, "Params:", params)

Best params: {'max_features': 10, 'min_samples_split': 60}
Score: 0.769921436588 Params: {'max_features': 2, 'min_samples_split': 2}
Score: 0.778900112233 Params: {'max_features': 2, 'min_samples_split': 4}
Score: 0.7822671156 Params: {'max_features': 2, 'min_samples_split': 6}
Score: 0.774410774411 Params: {'max_features': 2, 'min_samples_split': 8}
Score: 0.803591470258 Params: {'max_features': 2, 'min_samples_split': 10}
Score: 0.7822671156 Params: {'max_features': 2, 'min_samples_split': 20}
Score: 0.800224466891 Params: {'max_features': 2, 'min_samples_split': 40}
Score: 0.789001122334 Params: {'max_features': 2, 'min_samples_split': 60}
Score: 0.781144781145 Params: {'max_features': 2, 'min_samples_split': 80}
Score: 0.776655443322 Params: {'max_features': 3, 'min_samples_split': 2}
Score: 0.778900112233 Params: {'max_features': 3, 'min_samples_split': 4}
Score: 0.808080808081 Params: {'max_features': 3, 'min_samples_split': 6}
Score: 0.804713804714 Params: {'max_features': 3, 'm

In [10]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
param_grid = [
    {'solver': ['lbfgs'], 'alpha': [1e-6, 1e-7, 1e-8], 'hidden_layer_sizes': [(5, 2), (6, 2), (7, 3)], 'random_state': [1]}
]
mlp_classifier = MLPClassifier()
grid_search = GridSearchCV(mlp_classifier, param_grid, cv=3, scoring='accuracy')
grid_search.fit(X_train, y_train)
print("Best params:", grid_search.best_params_)

cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print("Score:", mean_score, "Params:", params)

Best params: {'alpha': 1e-06, 'hidden_layer_sizes': (5, 2), 'random_state': 1, 'solver': 'lbfgs'}
Score: 0.801346801347 Params: {'alpha': 1e-06, 'hidden_layer_sizes': (5, 2), 'random_state': 1, 'solver': 'lbfgs'}
Score: 0.793490460157 Params: {'alpha': 1e-06, 'hidden_layer_sizes': (6, 2), 'random_state': 1, 'solver': 'lbfgs'}
Score: 0.787878787879 Params: {'alpha': 1e-06, 'hidden_layer_sizes': (7, 3), 'random_state': 1, 'solver': 'lbfgs'}
Score: 0.800224466891 Params: {'alpha': 1e-07, 'hidden_layer_sizes': (5, 2), 'random_state': 1, 'solver': 'lbfgs'}
Score: 0.795735129068 Params: {'alpha': 1e-07, 'hidden_layer_sizes': (6, 2), 'random_state': 1, 'solver': 'lbfgs'}
Score: 0.792368125701 Params: {'alpha': 1e-07, 'hidden_layer_sizes': (7, 3), 'random_state': 1, 'solver': 'lbfgs'}
Score: 0.791245791246 Params: {'alpha': 1e-08, 'hidden_layer_sizes': (5, 2), 'random_state': 1, 'solver': 'lbfgs'}
Score: 0.79797979798 Params: {'alpha': 1e-08, 'hidden_layer_sizes': (6, 2), 'random_state': 1, 's

In [11]:
from sklearn.svm import SVC
param_grid = [
    {'C': [0.2, 0.5, 0.8, 1.0, 1.5, 2.0, 5.0, 10.0], 'kernel': ['poly'], 'degree': [2, 3, 4, 5, 7, 10], 'coef0': [0.8, 1.0, 1.2]},
]
sv_classifier = SVC()
grid_search = GridSearchCV(sv_classifier, param_grid, cv=10, scoring='accuracy')
grid_search.fit(X_train, y_train)
print("Best params:", grid_search.best_params_)

cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print("Score:", mean_score, "Params:", params)

Best params: {'C': 5.0, 'coef0': 1.2, 'degree': 3, 'kernel': 'poly'}
Score: 0.803591470258 Params: {'C': 0.2, 'coef0': 0.8, 'degree': 2, 'kernel': 'poly'}
Score: 0.820426487093 Params: {'C': 0.2, 'coef0': 0.8, 'degree': 3, 'kernel': 'poly'}
Score: 0.82379349046 Params: {'C': 0.2, 'coef0': 0.8, 'degree': 4, 'kernel': 'poly'}
Score: 0.822671156004 Params: {'C': 0.2, 'coef0': 0.8, 'degree': 5, 'kernel': 'poly'}
Score: 0.814814814815 Params: {'C': 0.2, 'coef0': 0.8, 'degree': 7, 'kernel': 'poly'}
Score: 0.800224466891 Params: {'C': 0.2, 'coef0': 0.8, 'degree': 10, 'kernel': 'poly'}
Score: 0.802469135802 Params: {'C': 0.2, 'coef0': 1.0, 'degree': 2, 'kernel': 'poly'}
Score: 0.820426487093 Params: {'C': 0.2, 'coef0': 1.0, 'degree': 3, 'kernel': 'poly'}
Score: 0.824915824916 Params: {'C': 0.2, 'coef0': 1.0, 'degree': 4, 'kernel': 'poly'}
Score: 0.82379349046 Params: {'C': 0.2, 'coef0': 1.0, 'degree': 5, 'kernel': 'poly'}
Score: 0.808080808081 Params: {'C': 0.2, 'coef0': 1.0, 'degree': 7, 'ker

In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
rf_clf = RandomForestClassifier(random_state=42)
cross_val_score(rf_clf, X_train, y_train, cv=3, scoring='accuracy').mean()

0.79236812570145909

In [13]:
X_test_prep = full_pipeline.transform(X_test)
sv_clf = SVC(C=2.0, kernel='poly')
sv_clf.fit(X_train, y_train)

SVC(C=2.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='poly',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [14]:
y_test = sv_clf.predict(X_test_prep)
df = pd.DataFrame(data={'PassengerId': X_test['PassengerId'], 'Survived': y_test})

In [15]:
df.to_csv('datasets/titanic/test_results.csv', index=False)